# Dimension = 30, Neighbors = 10, Search Space = 20% 

In [21]:
from scipy.optimize import minimize 
import matplotlib.pyplot as plt
import numpy as np 
import random
import scipy.optimize as opt

D = 30
N = 10 # neighboring point amount
func_num = 11

'''
FUNCTION TO BE OPTIMISED - Rastrigin
'''
def f(x):
    t = x
    t = 0.0512 * t
    tpi = 2.0 * np.pi
    sm = 0.0
    cs = np.cos(tpi*x)
    for i in range(0, len(x)):
        sm += t[i]*t[i] - 10*cs[i]
    return sm + 10*len(x)


'''
FUNCTION TO BE OPTIMISED - Rastrigin
'''
def g(x): # x: a list of neighboring solutions
    h = x
    for i in range(len(h)):
        h[i] = 0.0512 * h[i]
    #print(f"g:{x}")
    tpi = 2.0 * np.pi
    sm = 0.0
    cs = []
    for i in range(len(x)):
        cs.append(np.cos(tpi*h[i]))
    for i in range(0, len(x)):
        sm += h[i]*h[i] - 10*cs[i]
    return sm + 10*len(h)

'''
FUNCTION TO BE OPTIMISED - Rastrigin with average fitness
Put a list of points into the function to generate neighbors & calculate the average fitness value from generated neighbors
'''
def getNeighborAvg(x): 
# x: a list of new solutions
    values = [] # store the neighboring value 
    neighbors = []
    for _ in range(N):
        nb = []
        for j in range(D):
            xj = 0
            xj = x[j] + random.uniform(-20, 20) # 20% of the search area
            if (-100 > xj):
                xj = -100
            elif (100 < xj):
                xj = 100
            nb.append(xj)  
        neighbors.append(nb)
        
    for i in range(N): 
        value = g(neighbors[i])  # put the neighboring points into the objective function to get the fitness value 
        values.append(value) 
    avg = sum(values)/len(values) # the average fitness of the neighbors  
    return avg  

'''
Nelder Mead Implementation

- initial_simplex : array_like of shape (N + 1, N)
- maxiter, maxfev: Maximum allowed number of iterations and function evaluations.
- xatol : float
        Absolute error in xopt between iterations that is acceptable for
        convergence.
- fatol : number
        Absolute error in func(xopt) between iterations that is acceptable for
        convergence.

Result contains the following attributes:
    fun - value of objective function 
    nfev -  the number of evaluation, 
    nit - the number of iterations, 
    success - the existence of optimizer, 
    x -  the solution of the optimization,
    status -  termination status
'''
def BFGS(fun,x0): # fun: function; x0: random start point (N,1)
    maxiter, maxfev, xatol, fatol = N*20000, N*20000, 1e-5, 1e-5 # default values: N*200, 1e-4
    result1 = opt.minimize(fun, x0, method='BFGS', options={ 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
    print(f"value {result1['fun']},  \n{result1['message']}, \n{result1['nfev']} evaluations \n{result1['nit']} iterations") 
    #print(result1.x) # the position of optimal value 
    
'''
Nelder Mead with Average Fitness
'''
def avgBFGS(fun,x0): # f: function; x0: start point
    maxiter, maxfev, xatol, fatol = N*20000, N*20000, 1e-5, 1e-5 # default values: N*200, 1e-4
    step = 0
    # simplex: (N + 1, N)
    simplex = np.vstack((x0,x0+np.diag((step,step,step,step,step,step,step,step,step,step,
                                        step,step,step,step,step,step,step,step,step,step,
                                       step,step,step,step,step,step,step,step,step,step,)))) 
    # adjust every element in simplex to make it bigger
    point = []
    for i in range(len(simplex)):
        for j in range(D):
            simplex[i][j] = simplex[i][j] + random.uniform(-10, 10)
            simplex[i][j] = simplex[i][j] + random.choice([-20, 20])
            if (-100 > simplex[i][j]):
                simplex[i][j] = -100
            elif (100 < simplex[i][j]):
                simplex[i][j] = 100
    #print(simplex)
    result2 = opt.minimize(fun, x0, method='BFGS', options={'initial_simplex':simplex, 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
    # After finishing each trial of Nelder-Mead with average fitness, its final point is used for the standard Nelder-Mead ???
    # the position of optimal value: result2.x
    # use the default simplex
    result2 = opt.minimize(f, result2.x, method='BFGS', options={'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
    print(f"value {result2['fun']},  \n{result2['message']}, \n{result2['nfev']} evaluations \n{result2['nit']} iterations") 
    #print(result2.x) 


"""
Test 
"""
if __name__ == "__main__":
    for i in range(1,31):
        # get the initial point
        x0 = 0 # start point
        space = np.array([[-100, 100]])
        bounds = space
        for _ in range(D-1):
            bounds = np.append(bounds,space)
        bounds = bounds.reshape(D,2)
        lower_bounds, upper_bounds = bounds[:, 0], bounds[:, 1]
        for _ in range(N): # N random restarts 
            x0 = lower_bounds + (np.random.rand(1) * (upper_bounds - lower_bounds))  # random start point
        
        x_center = x0
        print(f"\nBFGS: run {i}")   
        BFGS(f,x_center)
        print(f"\nBFGS with average fitness: run {i}")   
        avgBFGS(getNeighborAvg,x0)


Nelder Mead: run 1
value 541.765809920734,  
Optimization terminated successfully., 
310 evaluations 
5 iterations

Nelder Mead with average fitness: run 1


/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:90: OptimizeWarning: Unknown solver options: maxfev, xatol, fatol
  result1 = opt.minimize(fun, x0, method='BFGS', options={ 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:115: OptimizeWarning: Unknown solver options: initial_simplex, maxfev, xatol, fatol
  result2 = opt.minimize(fun, x0, method='BFGS', options={'initial_simplex':simplex, 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:119: OptimizeWarning: Unknown solver options: maxfev, xatol, fatol
  result2 = opt.minimize(f, result2.x, method='BFGS', options={'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})


value 541.765809920734,  
Optimization terminated successfully., 
310 evaluations 
5 iterations

Nelder Mead: run 2
value 515.9711828856032,  
Optimization terminated successfully., 
217 evaluations 
5 iterations

Nelder Mead with average fitness: run 2
value 515.9711828856047,  
Optimization terminated successfully., 
2728 evaluations 
55 iterations

Nelder Mead: run 3
value 237.8925206937169,  
Optimization terminated successfully., 
403 evaluations 
8 iterations

Nelder Mead with average fitness: run 3


/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:90: OptimizeWarning: Unknown solver options: maxfev, xatol, fatol
  result1 = opt.minimize(fun, x0, method='BFGS', options={ 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:115: OptimizeWarning: Unknown solver options: initial_simplex, maxfev, xatol, fatol
  result2 = opt.minimize(fun, x0, method='BFGS', options={'initial_simplex':simplex, 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:119: OptimizeWarning: Unknown solver options: maxfev, xatol, fatol
  result2 = opt.minimize(f, result2.x, method='BFGS', options={'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:90: OptimizeWarning: Unknown solver options: maxfev, xatol, fat

value 237.8925206937223,  
Desired error not necessarily achieved due to precision loss., 
1767 evaluations 
30 iterations

Nelder Mead: run 4
value 1.966053890080616,  
Desired error not necessarily achieved due to precision loss., 
2046 evaluations 
39 iterations

Nelder Mead with average fitness: run 4
value 1.9660538900772622,  
Optimization terminated successfully., 
2511 evaluations 
51 iterations

Nelder Mead: run 5
value 229.32052573337563,  
Optimization terminated successfully., 
2666 evaluations 
49 iterations

Nelder Mead with average fitness: run 5


...

value 0.7077794004281941,  
Optimization terminated successfully., 
2604 evaluations 
53 iterations

Nelder Mead: run 29
value 0.31456862241259387,  
Optimization terminated successfully., 
1457 evaluations 
21 iterations

Nelder Mead with average fitness: run 29


/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:90: OptimizeWarning: Unknown solver options: maxfev, xatol, fatol
  result1 = opt.minimize(fun, x0, method='BFGS', options={ 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:115: OptimizeWarning: Unknown solver options: initial_simplex, maxfev, xatol, fatol
  result2 = opt.minimize(fun, x0, method='BFGS', options={'initial_simplex':simplex, 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:119: OptimizeWarning: Unknown solver options: maxfev, xatol, fatol
  result2 = opt.minimize(f, result2.x, method='BFGS', options={'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})


value 0.31456862241282124,  
Optimization terminated successfully., 
2604 evaluations 
51 iterations

Nelder Mead: run 30
value 9.515700827962007,  
Optimization terminated successfully., 
961 evaluations 
19 iterations

Nelder Mead with average fitness: run 30


/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:90: OptimizeWarning: Unknown solver options: maxfev, xatol, fatol
  result1 = opt.minimize(fun, x0, method='BFGS', options={ 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:115: OptimizeWarning: Unknown solver options: initial_simplex, maxfev, xatol, fatol
  result2 = opt.minimize(fun, x0, method='BFGS', options={'initial_simplex':simplex, 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
/var/folders/3f/0c48rx816935875jnxcqj5sr0000gn/T/ipykernel_5133/1526502344.py:119: OptimizeWarning: Unknown solver options: maxfev, xatol, fatol
  result2 = opt.minimize(f, result2.x, method='BFGS', options={'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})


value 9.515700827968317,  
Desired error not necessarily achieved due to precision loss., 
775 evaluations 
12 iterations


# Sphere test

In [13]:
from scipy.optimize import minimize 
import matplotlib.pyplot as plt
import numpy as np 
import random
import scipy.optimize as opt

D = 30
N = 10 # neighboring point amount
func_num = 11

In [14]:
'''
FUNCTION TO BE OPTIMISED - Sphere
'''
def sphere(x):
    x = 0.0512 * x
    res = 0
    for i in range(len(x)):
        res += x[i]*x[i]
    return res


def sphere_neighbor(x): # x: a list of neighboring solutions
    h = x
    for i in range(len(h)):
        h[i] = 0.0512 * h[i]
    res = 0
    for i in range(len(x)):
        res += x[i]*x[i]
    return res

In [16]:
'''
FUNCTION TO BE OPTIMISED with average fitness
Put a list of points into the function to generate neighbors & calculate the average fitness value from generated neighbors
'''
def getNeighborAvg(x): 
# x: a list of new solutions
    values = [] # store the neighboring value 
    neighbors = []
    for _ in range(N):
        nb = []
        for j in range(D):
            xj = 0
            xj = x[j] + random.uniform(-20, 20) # 20% of the search area
            if (-100 > xj):
                xj = -100
            elif (100 < xj):
                xj = 100
            nb.append(xj)  
        neighbors.append(nb)
        
    for i in range(N): 
        value = sphere_neighbor(neighbors[i])  # put the neighboring points into the objective function to get the fitness value 
        values.append(value) 
    avg = sum(values)/len(values) # the average fitness of the neighbors  
    return avg  

'''
Nelder Mead Implementation

- initial_simplex : array_like of shape (N + 1, N)
- maxiter, maxfev: Maximum allowed number of iterations and function evaluations.
- xatol : float
        Absolute error in xopt between iterations that is acceptable for
        convergence.
- fatol : number
        Absolute error in func(xopt) between iterations that is acceptable for
        convergence.

Result contains the following attributes:
    fun - value of objective function 
    nfev -  the number of evaluation, 
    nit - the number of iterations, 
    success - the existence of optimizer, 
    x -  the solution of the optimization,
    status -  termination status
'''
def nelder(fun,x0): # fun: function; x0: random start point (N,1)
    maxiter, maxfev, xatol, fatol = N*20000, N*20000, 1e-5, 1e-5 # default values: N*200, 1e-4
    result1 = opt.minimize(fun, x0, method='Nelder-Mead', options={ 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
    print(f"value {result1['fun']},  \n{result1['message']}, \n{result1['nfev']} evaluations \n{result1['nit']} iterations") 
    #print(result1.x) # the position of optimal value 
    
'''
Nelder Mead with Average Fitness
'''
def nelderFavg(fun,x0): # f: function; x0: start point
    maxiter, maxfev, xatol, fatol = N*20000, N*20000, 1e-5, 1e-5 # default values: N*200, 1e-4
    step = 0
    # simplex: (N + 1, N)
    simplex = np.vstack((x0,x0+np.diag((step,step,step,step,step,step,step,step,step,step,
                                        step,step,step,step,step,step,step,step,step,step,
                                       step,step,step,step,step,step,step,step,step,step,)))) 
    # adjust every element in simplex to make it bigger
    point = []
    for i in range(len(simplex)):
        for j in range(D):
            simplex[i][j] = simplex[i][j] + random.uniform(-10, 10)
            simplex[i][j] = simplex[i][j] + random.choice([-20, 20])
            if (-100 > simplex[i][j]):
                simplex[i][j] = -100
            elif (100 < simplex[i][j]):
                simplex[i][j] = 100
    #print(simplex)
    result2 = opt.minimize(fun, x0, method='Nelder-Mead', options={'initial_simplex':simplex, 'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
    # After finishing each trial of Nelder-Mead with average fitness, its final point is used for the standard Nelder-Mead ???
    # the position of optimal value: result2.x
    # use the default simplex
    result2 = opt.minimize(sphere, result2.x, method='Nelder-Mead', options={'maxiter':maxiter, 'maxfev':maxfev, 'xatol':xatol, 'fatol':fatol})
    print(f"value {result2['fun']},  \n{result2['message']}, \n{result2['nfev']} evaluations \n{result2['nit']} iterations") 
    #print(result2.x) 


"""
Test 
"""
if __name__ == "__main__":
    for i in range(1,31):
        # get the initial point
        x0 = 0 # start point
        space = np.array([[-100, 100]])
        bounds = space
        for _ in range(D-1):
            bounds = np.append(bounds,space)
        bounds = bounds.reshape(D,2)
        lower_bounds, upper_bounds = bounds[:, 0], bounds[:, 1]
        for _ in range(N): # N random restarts 
            x0 = lower_bounds + (np.random.rand(1) * (upper_bounds - lower_bounds))  # random start point
        
        x_center = x0
        print(f"\nNelder Mead: run {i}")   
        nelder(sphere,x_center)
        print(f"\nNelder Mead with average fitness: run {i}")   
        nelderFavg(getNeighborAvg,x0)


Nelder Mead: run 1
value 3.056082021612129e-09,  
Optimization terminated successfully., 
51505 evaluations 
43478 iterations

Nelder Mead with average fitness: run 1
value 3.056091381589454,  
Optimization terminated successfully., 
72868 evaluations 
60795 iterations

Nelder Mead: run 2
value 2.8632205638211505e-10,  
Optimization terminated successfully., 
48615 evaluations 
41030 iterations

Nelder Mead with average fitness: run 2
value 0.6620412307783377,  
Optimization terminated successfully., 
73770 evaluations 
61706 iterations

Nelder Mead: run 3
value 1.864667030486179e-09,  
Optimization terminated successfully., 
51322 evaluations 
43329 iterations

Nelder Mead with average fitness: run 3
value 1.5364466744817111,  
Optimization terminated successfully., 
84296 evaluations 
70734 iterations

Nelder Mead: run 4
value 8.070665675112311e-10,  
Optimization terminated successfully., 
50475 evaluations 
42609 iterations

Nelder Mead with average fitness: run 4
value 0.38177299

In [68]:
def _minimize_neldermead(func, x0, args=(), callback=None,
                         maxiter=None, maxfev=None, disp=False,
                         return_all=False, initial_simplex=None,
                         xatol=1e-4, fatol=1e-4, adaptive=False, bounds=None,
                         **unknown_options):
    """
    Minimization of scalar function of one or more variables using the
    Nelder-Mead algorithm.
    Options
    -------
    disp : bool
        Set to True to print convergence messages.
    maxiter, maxfev : int
        Maximum allowed number of iterations and function evaluations.
        Will default to ``N*200``, where ``N`` is the number of
        variables, if neither `maxiter` or `maxfev` is set. If both
        `maxiter` and `maxfev` are set, minimization will stop at the
        first reached.
    return_all : bool, optional
        Set to True to return a list of the best solution at each of the
        iterations.
    initial_simplex : array_like of shape (N + 1, N)
        Initial simplex. If given, overrides `x0`.
        ``initial_simplex[j,:]`` should contain the coordinates of
        the jth vertex of the ``N+1`` vertices in the simplex, where
        ``N`` is the dimension.
    xatol : float, optional
        Absolute error in xopt between iterations that is acceptable for
        convergence.
    fatol : number, optional
        Absolute error in func(xopt) between iterations that is acceptable for
        convergence.
    adaptive : bool, optional
        Adapt algorithm parameters to dimensionality of problem. Useful for
        high-dimensional minimization [1]_.
    bounds : sequence or `Bounds`, optional
        Bounds on variables. There are two ways to specify the bounds:
            1. Instance of `Bounds` class.
            2. Sequence of ``(min, max)`` pairs for each element in `x`. None
               is used to specify no bound.
        Note that this just clips all vertices in simplex based on
        the bounds.
    References
    ----------
    .. [1] Gao, F. and Han, L.
       Implementing the Nelder-Mead simplex algorithm with adaptive
       parameters. 2012. Computational Optimization and Applications.
       51:1, pp. 259-277
    """
    _check_unknown_options(unknown_options)
    maxfun = maxfev
    retall = return_all

    x0 = asfarray(x0).flatten()

    if adaptive:
        dim = float(len(x0))
        rho = 1
        chi = 1 + 2/dim
        psi = 0.75 - 1/(2*dim)
        sigma = 1 - 1/dim
    else:
        rho = 1
        chi = 2
        psi = 0.5
        sigma = 0.5

    nonzdelt = 0.05
    zdelt = 0.00025

    if bounds is not None:
        lower_bound, upper_bound = bounds.lb, bounds.ub
        # check bounds
        if (lower_bound > upper_bound).any():
            raise ValueError("Nelder Mead - one of the lower bounds is greater than an upper bound.")
        if np.any(lower_bound > x0) or np.any(x0 > upper_bound):
            warnings.warn("Initial guess is not within the specified bounds",
                          OptimizeWarning, 3)

    if bounds is not None:
        x0 = np.clip(x0, lower_bound, upper_bound)

    if initial_simplex is None:
        N = len(x0)

        sim = np.empty((N + 1, N), dtype=x0.dtype)
        sim[0] = x0
        for k in range(N):
            y = np.array(x0, copy=True)
            if y[k] != 0:
                y[k] = (1 + nonzdelt)*y[k]
            else:
                y[k] = zdelt
            sim[k + 1] = y
    else:
        sim = np.asfarray(initial_simplex).copy()
        if sim.ndim != 2 or sim.shape[0] != sim.shape[1] + 1:
            raise ValueError("`initial_simplex` should be an array of shape (N+1,N)")
        if len(x0) != sim.shape[1]:
            raise ValueError("Size of `initial_simplex` is not consistent with `x0`")
        N = sim.shape[1]

    if retall:
        allvecs = [sim[0]]

    # If neither are set, then set both to default
    if maxiter is None and maxfun is None:
        maxiter = N * 200
        maxfun = N * 200
    elif maxiter is None:
        # Convert remaining Nones, to np.inf, unless the other is np.inf, in
        # which case use the default to avoid unbounded iteration
        if maxfun == np.inf:
            maxiter = N * 200
        else:
            maxiter = np.inf
    elif maxfun is None:
        if maxiter == np.inf:
            maxfun = N * 200
        else:
            maxfun = np.inf

    if bounds is not None:
        sim = np.clip(sim, lower_bound, upper_bound)

    one2np1 = list(range(1, N + 1))
    fsim = np.full((N + 1,), np.inf, dtype=float)

    fcalls, func = _wrap_scalar_function_maxfun_validation(func, args, maxfun)

    try:
        for k in range(N + 1):
            fsim[k] = func(sim[k])
    except _MaxFuncCallError:
        pass
    finally:
        ind = np.argsort(fsim)
        sim = np.take(sim, ind, 0)
        fsim = np.take(fsim, ind, 0)

    ind = np.argsort(fsim)
    fsim = np.take(fsim, ind, 0)
    # sort so sim[0,:] has the lowest function value
    sim = np.take(sim, ind, 0)

    iterations = 1

    while (fcalls[0] < maxfun and iterations < maxiter):
        try:
            if (np.max(np.ravel(np.abs(sim[1:] - sim[0]))) <= xatol and
                    np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):
                break

            xbar = np.add.reduce(sim[:-1], 0) / N
            xr = (1 + rho) * xbar - rho * sim[-1]
            if bounds is not None:
                xr = np.clip(xr, lower_bound, upper_bound)
            fxr = func(xr)
            doshrink = 0

            if fxr < fsim[0]:
                xe = (1 + rho * chi) * xbar - rho * chi * sim[-1]
                if bounds is not None:
                    xe = np.clip(xe, lower_bound, upper_bound)
                fxe = func(xe)

                if fxe < fxr:
                    sim[-1] = xe
                    fsim[-1] = fxe
                else:
                    sim[-1] = xr
                    fsim[-1] = fxr
            else:  # fsim[0] <= fxr
                if fxr < fsim[-2]:
                    sim[-1] = xr
                    fsim[-1] = fxr
                else:  # fxr >= fsim[-2]
                    # Perform contraction
                    if fxr < fsim[-1]:
                        xc = (1 + psi * rho) * xbar - psi * rho * sim[-1]
                        if bounds is not None:
                            xc = np.clip(xc, lower_bound, upper_bound)
                        fxc = func(xc)

                        if fxc <= fxr:
                            sim[-1] = xc
                            fsim[-1] = fxc
                        else:
                            doshrink = 1
                    else:
                        # Perform an inside contraction
                        xcc = (1 - psi) * xbar + psi * sim[-1]
                        if bounds is not None:
                            xcc = np.clip(xcc, lower_bound, upper_bound)
                        fxcc = func(xcc)

                        if fxcc < fsim[-1]:
                            sim[-1] = xcc
                            fsim[-1] = fxcc
                        else:
                            doshrink = 1

                    if doshrink:
                        for j in one2np1:
                            sim[j] = sim[0] + sigma * (sim[j] - sim[0])
                            if bounds is not None:
                                sim[j] = np.clip(
                                    sim[j], lower_bound, upper_bound)
                            fsim[j] = func(sim[j])
            iterations += 1
        except _MaxFuncCallError:
            pass
        finally:
            ind = np.argsort(fsim)
            sim = np.take(sim, ind, 0)
            fsim = np.take(fsim, ind, 0)
            if callback is not None:
                callback(sim[0])
            if retall:
                allvecs.append(sim[0])

    x = sim[0]
    fval = np.min(fsim)
    warnflag = 0

    if fcalls[0] >= maxfun:
        warnflag = 1
        msg = _status_message['maxfev']
        if disp:
            warnings.warn(msg, RuntimeWarning, 3)
    elif iterations >= maxiter:
        warnflag = 2
        msg = _status_message['maxiter']
        if disp:
            warnings.warn(msg, RuntimeWarning, 3)
    else:
        msg = _status_message['success']
        if disp:
            print(msg)
            print("         Current function value: %f" % fval)
            print("         Iterations: %d" % iterations)
            print("         Function evaluations: %d" % fcalls[0])

    result = OptimizeResult(fun=fval, nit=iterations, nfev=fcalls[0],
                            status=warnflag, success=(warnflag == 0),
                            message=msg, x=x, final_simplex=(sim, fsim))
    if retall:
        result['allvecs'] = allvecs
    return result

In [20]:
H = np.eye(2)*2
H

array([[2., 0.],
       [0., 2.]])